In [1]:
from mxnet import np,npx
from mxnet import autograd

npx.set_np()

In [2]:
x=np.array([[1,2],[3,4]])
x

[20:44:31] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


array([[1., 2.],
       [3., 4.]])

In [3]:
x.attach_grad()

In [5]:
with autograd.record():
    y=2*x**2

In [6]:
y.backward()

Derivada de y=2x² => 4x

In [8]:
x.grad,y

(array([[ 4.,  8.],
        [12., 16.]]),
 array([[ 2.,  8.],
        [18., 32.]]))

In [9]:
with autograd.record():
    y=np.sum(2*x**2)
y.backward
x.grad

array([[ 4.,  8.],
       [12., 16.]])

In [10]:
with autograd.record():
    y=np.sum(2*x**2)
y.backward(retain_graph=True)
print(x.grad)
print("backward again because grad ratain")
y.backward()
print(x.grad)

try:
    y.backward()
except:
    print("não tem grad")

[[ 4.  8.]
 [12. 16.]]
backward again because grad ratain
[[ 4.  8.]
 [12. 16.]]
não tem grad


Custom MXNet ndarray operations